In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime

In [0]:
dbutils.fs.ls('/FileStore/')

[FileInfo(path='dbfs:/FileStore/2024_04_18_4_WheelSales.csv', name='2024_04_18_4_WheelSales.csv', size=10992762, modificationTime=1719989742000),
 FileInfo(path='dbfs:/FileStore/4_WheelSales.csv', name='4_WheelSales.csv', size=10992762, modificationTime=1719989774000),
 FileInfo(path='dbfs:/FileStore/jars/', name='jars/', size=0, modificationTime=1716203553000),
 FileInfo(path='dbfs:/FileStore/source1.csv', name='source1.csv', size=1138, modificationTime=1709555634000),
 FileInfo(path='dbfs:/FileStore/tables/', name='tables/', size=0, modificationTime=1708600418000)]

In [0]:
df = spark.read.format('csv').option('Header', True).option('inferSchema', True).load('dbfs:/FileStore/2024_04_18_4_WheelSales-1.csv')



In [0]:
print(df.columns)

['Brand', 'Model', 'Body Type', 'FUEL TYPE', 'Mileage', 'ENGINE', 'TRANSMISSION', 'Price range', 'Fuel Capacity (L)', 'Seating Capacity', 'Top_Speed in Km/h', 'No Of Cylinders', 'Year Of Sale', 'Month Of Sale', 'Country', 'City']


In [0]:
schemas = df.dtypes

types_in_list = [x[1] for x in schemas]

print(types_in_list)

['string', 'string', 'string', 'string', 'string', 'string', 'string', 'string', 'int', 'int', 'string', 'int', 'int', 'string', 'string', 'string']


In [0]:
dbutils.secrets.list(scope='adf-adb-generic')


[SecretMetadata(key='accountkey'),
 SecretMetadata(key='accountname'),
 SecretMetadata(key='blodstorelogs'),
 SecretMetadata(key='clientid'),
 SecretMetadata(key='fb-app-id'),
 SecretMetadata(key='fb-app-secret'),
 SecretMetadata(key='fb-longlived-accesstoken'),
 SecretMetadata(key='Gai-access-token'),
 SecretMetadata(key='Gai-client-id'),
 SecretMetadata(key='Gai-client-secret'),
 SecretMetadata(key='Gai-refresh-token'),
 SecretMetadata(key='insta-client-id'),
 SecretMetadata(key='insta-client-secret'),
 SecretMetadata(key='insta-longlived-access-token'),
 SecretMetadata(key='insta-shortlive-access-token'),
 SecretMetadata(key='password'),
 SecretMetadata(key='refreshtokenxero'),
 SecretMetadata(key='secretvalue'),
 SecretMetadata(key='smtpport'),
 SecretMetadata(key='smtpserver'),
 SecretMetadata(key='smtpuser'),
 SecretMetadata(key='tenantid'),
 SecretMetadata(key='xero-access-token'),
 SecretMetadata(key='xero-client-id'),
 SecretMetadata(key='xero-client-seceret'),
 SecretMetadata

In [0]:
jdbcHostname = "prospectmanagement.database.windows.net"
jdbcDatabase = "ProspectManagement"
jdbcPort = 1433
jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)
connectionProperties = {
"user" : "ProspectManagementAdmin@prospectmanagement",
"password" : "Admin@123",
"driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [0]:
control_df = spark.read.jdbc(url=jdbcUrl, table='dev.v_test',properties=connectionProperties)

In [0]:
print(control_df.columns)

['PipelineName', 'Pk_PipelineID', 'Pk_ActivityID', 'PipelineSourceTypes', 'ParameterValueSink', 'ColumnSourceTypes', 'SourceColumnName', 'TargetColumnName', 'sourcecolumntypes', 'IsNullable', 'IsActive']


In [0]:
display(control_df)

PipelineName,Pk_PipelineID,Pk_ActivityID,PipelineSourceTypes,ParameterValueSink,ColumnSourceTypes,SourceColumnName,TargetColumnName,sourcecolumntypes,IsNullable,IsActive
PL_Copy2WheelsData,101,2,CSV,2-WheelSales.csv,CSV,"[OEM, Model, Segment, Month, Year, NoOfSales, EngineCapacity, Mileage, Model Price, Dealer location, Region]","[OEM, Model, Segment, Month, Year, NoOfSales, EngineCapacity, Mileage, ModelPrice, Dealerlocation, Region]","[string, string, string, string, int, int, string, string, string, string, string]",0,1
PL_Copy4WheelsData,102,3,CSV,4-WheelSales.csv,CSV,"[Brand, Model, Body Type, FUEL TYPE, Mileage, ENGINE, TRANSMISSION, Price range, Fuel Capacity (L), Seating Capacity, Top_Speed in Km/h, No Of Cylinders, Year Of Sale, Month Of Sale , Country, City]","[Brand, Model, BodyType, FUELTYPE, Mileage, ENGINE, TRANSMISSION, Pricerange, FuelCapacity, SeatingCapacity, Top_Speed, NoOfCylinders, YearOfSale, MonthOfSale, Country, City]","[string, string, string, string, string, string, string, string, int, int, string, int, int, string, string, string]",null,null


In [0]:
res = control_df.withColumn('test', split(col('ParameterValueSink'), '.csv').getItem(0))

display(res)

PipelineName,Pk_PipelineID,Pk_ActivityID,PipelineSourceTypes,ParameterValueSink,ColumnSourceTypes,SourceColumnName,TargetColumnName,sourcecolumntypes,IsNullable,IsActive,test
PL_Copy2WheelsData,101,2,CSV,2-WheelSales.csv,CSV,"[OEM, Model, Segment, Month, Year, NoOfSales, EngineCapacity, Mileage, Model Price, Dealer location, Region]","[OEM, Model, Segment, Month, Year, NoOfSales, EngineCapacity, Mileage, ModelPrice, Dealerlocation, Region]","[string, string, string, string, int, int, string, string, string, string, string]",0,1,2-WheelSales
PL_Copy4WheelsData,102,3,CSV,4-WheelSales.csv,CSV,"[Brand, Model, Body Type, FUEL TYPE, Mileage, ENGINE, TRANSMISSION, Price range, Fuel Capacity (L), Seating Capacity, Top_Speed in Km/h, No Of Cylinders, Year Of Sale, Month Of Sale , Country, City]","[Brand, Model, BodyType, FUELTYPE, Mileage, ENGINE, TRANSMISSION, Pricerange, FuelCapacity, SeatingCapacity, Top_Speed, NoOfCylinders, YearOfSale, MonthOfSale, Country, City]","[string, string, string, string, string, string, string, string, int, int, string, int, int, string, string, string]",null,null,4-WheelSales


In [0]:
print(res)

[Row(ParameterValueSink='2-WheelSData.csv'), Row(ParameterValueSink='4-WheelSData.csv')]


In [0]:
def get_secrets():
    try:
        
        application_id = dbutils.secrets.get(scope='adf-adb-generic', key='clientid')
        authenticationKey = dbutils.secrets.get(scope='adf-adb-generic', key='secretvalue')
        tenant_id = dbutils.secrets.get(scope='adf-adb-generic', key='tenantid')

        return application_id, authenticationKey, tenant_id
    
    except Exception as e :
        print(f"An error occure in get_secrets method:: {str(e)}")
        raise e 

In [0]:
def mount_raw(adls_container_name, storage_account_name, adls_folder_name, mount_point):
    try:
        dt_string = datetime.now().strftime("year=%Y/Month=%m/day=%d")
            
        application_id, authenticationKey, tenant_id = get_secrets()
        
        #formated_path = f"year = {now.strftime('%Y')}/month = {now.strftime('%m')}/day = {now.strftime('%d')}"
    
        #date_folder_path = f"{adls_folder_name}/{dt_string}"
        
        source = f"abfss://{adls_container_name}@{storage_account_name}.dfs.core.windows.net/{adls_folder_name}/"
        print(source)
        endpoint = f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"

        configs = {
            "fs.azure.account.auth.type": "OAuth",
            "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
            "fs.azure.account.oauth2.client.id": f"{application_id}",
            "fs.azure.account.oauth2.client.secret": f"{authenticationKey}",
            "fs.azure.account.oauth2.client.endpoint": f"{endpoint}",
            "fs.azure.createRemoteFileSystemDuringInitialization": "true"
        }

        if not any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
            dbutils.fs.mount(
                source=source,
                mount_point=mount_point,
                extra_configs=configs
            )
            
        
    except Exception as e:
        print(f"An error occure in mount_adls_folde method:: {str(e)}")
        raise e 


In [0]:
def mount_stage(adls_container_name, storage_account_name, adls_folder_name, mount_point):
    try:
        dt_string = datetime.now().strftime("year=%Y/Month=%m/day=%d")
            
        application_id, authenticationKey, tenant_id = get_secrets()
        
        #formated_path = f"year = {now.strftime('%Y')}/month = {now.strftime('%m')}/day = {now.strftime('%d')}"
    
        #date_folder_path = f"{adls_folder_name}/{dt_string}"
        
        source = f"abfss://{adls_container_name}@{storage_account_name}.dfs.core.windows.net/{adls_folder_name}/"
        print(source)
        endpoint = f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"

        configs = {
            "fs.azure.account.auth.type": "OAuth",
            "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
            "fs.azure.account.oauth2.client.id": f"{application_id}",
            "fs.azure.account.oauth2.client.secret": f"{authenticationKey}",
            "fs.azure.account.oauth2.client.endpoint": f"{endpoint}",
            "fs.azure.createRemoteFileSystemDuringInitialization": "true"
        }

        if not any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
            dbutils.fs.mount(
                source=source,
                mount_point=mount_point,
                extra_configs=configs
            )
            
        
    except Exception as e:
        print(f"An error occure in mount_adls_folde method:: {str(e)}")
        raise e 


In [0]:
def mount_silver(adls_container_name, storage_account_name, adls_folder_name, mount_point):
    try:
        dt_string = datetime.now().strftime("year=%Y/Month=%m/day=%d")
            
        application_id, authenticationKey, tenant_id = get_secrets()
        
        #formated_path = f"year = {now.strftime('%Y')}/month = {now.strftime('%m')}/day = {now.strftime('%d')}"
    
        #date_folder_path = f"{adls_folder_name}/{dt_string}"
        
        source = f"abfss://{adls_container_name}@{storage_account_name}.dfs.core.windows.net/{adls_folder_name}/"
        print(source)
        endpoint = f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"

        configs = {
            "fs.azure.account.auth.type": "OAuth",
            "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
            "fs.azure.account.oauth2.client.id": f"{application_id}",
            "fs.azure.account.oauth2.client.secret": f"{authenticationKey}",
            "fs.azure.account.oauth2.client.endpoint": f"{endpoint}",
            "fs.azure.createRemoteFileSystemDuringInitialization": "true"
        }

        if not any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
            dbutils.fs.mount(
                source=source,
                mount_point=mount_point,
                extra_configs=configs
            )
            
        
    except Exception as e:
        print(f"An error occure in mount_adls_folde method:: {str(e)}")
        raise e 


In [0]:
## for silver mount points 

for row in res.collect():
    adlsContainerName = 'xyenta-dev-96-silver'
    storageAccountName = 'adlsxyentadevuks96'
    mountPointSuffix = row['test']
    mountPoint = f"/mnt/genericSilver/{mountPointSuffix}"
    print(mountPointSuffix)
    print(mountPoint)

    mount_silver(adls_container_name= adlsContainerName, storage_account_name= storageAccountName, adls_folder_name= mountPointSuffix, mount_point= mountPoint)


2-WheelSales
/mnt/genericSilver/2-WheelSales
abfss://xyenta-dev-96-silver@adlsxyentadevuks96.dfs.core.windows.net/2-WheelSales/
4-WheelSales
/mnt/genericSilver/4-WheelSales
abfss://xyenta-dev-96-silver@adlsxyentadevuks96.dfs.core.windows.net/4-WheelSales/


In [0]:

for row in res.collect():
    adlsContainerName = 'xyenta-dev-96-raw'
    storageAccountName = 'adlsxyentadevuks96'
    mountPointSuffix = row['test']
    # mountPointSuffix = 'Test'
    mountPoint = f"/mnt/genericrawTest/{mountPointSuffix}"
    print(mountPointSuffix)
    print(mountPoint)

    mount_raw(adls_container_name= adlsContainerName, storage_account_name= storageAccountName, adls_folder_name= mountPointSuffix, mount_point= mountPoint)

Test
/mnt/genericrawTest/Test
abfss://xyenta-dev-96-raw@adlsxyentadevuks96.dfs.core.windows.net/Test/
Test
/mnt/genericrawTest/Test
abfss://xyenta-dev-96-raw@adlsxyentadevuks96.dfs.core.windows.net/Test/


In [0]:
df_p.printSchema()

root
 |-- OEM: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- NoOfSales: string (nullable = true)
 |-- EngineCapacity: string (nullable = true)
 |-- Mileage: string (nullable = true)
 |-- ModelPrice: string (nullable = true)
 |-- Dealerlocation: string (nullable = true)
 |-- Region: string (nullable = true)



In [0]:
# adlsContainerName = 'xyenta-dev-96-raw'
# storageAccountName = 'adlsxyentadevuks96'

for row in res.collect():
    adlsContainerName = 'xyenta-dev-96-stg'
    storageAccountName = 'adlsxyentadevuks96'
    mountPointSuffix = row['test']
    mountPoint = f"/mnt/genericStage/{mountPointSuffix}"
    print(mountPointSuffix)
    print(mountPoint)

    mount_stage(adls_container_name= adlsContainerName, storage_account_name= storageAccountName, adls_folder_name= mountPointSuffix, mount_point= mountPoint)



2-WheelSales
/mnt/genericStage/2-WheelSales
abfss://xyenta-dev-96-stg@adlsxyentadevuks96.dfs.core.windows.net/2-WheelSales/
4-WheelSales
/mnt/genericStage/4-WheelSales
abfss://xyenta-dev-96-stg@adlsxyentadevuks96.dfs.core.windows.net/4-WheelSales/


In [0]:
# dbutils.fs.mounts()

[MountInfo(mountPoint='/mnt/genericrawTest/Test', source='abfss://xyenta-dev-96-raw@adlsxyentadevuks96.dfs.core.windows.net/Test/', encryptionType=''),
 MountInfo(mountPoint='/mnt/genericraw/2-WheelSales', source='abfss://xyenta-dev-96-raw@adlsxyentadevuks96.dfs.core.windows.net/2-WheelSales/', encryptionType=''),
 MountInfo(mountPoint='/mnt/StageDev/s', source='abfss://xyenta-dev-96-stg@adlsxyentadevuks96.dfs.core.windows.net/s/year=2024/month=01/day=06/', encryptionType=''),
 MountInfo(mountPoint='/mnt/StageDev/d', source='abfss://xyenta-dev-96-stg@adlsxyentadevuks96.dfs.core.windows.net/d/year=2024/month=01/day=06/', encryptionType=''),
 MountInfo(mountPoint='/databricks-datasets', source='databricks-datasets', encryptionType=''),
 MountInfo(mountPoint='/mnt/StageDev/O', source='abfss://xyenta-dev-96-stg@adlsxyentadevuks96.dfs.core.windows.net/O/year=2024/month=01/day=06/', encryptionType=''),
 MountInfo(mountPoint='/Volumes', source='UnityCatalogVolumes', encryptionType=''),
 Mount

In [0]:
## /mnt/genericSilver/
## /mnt/genericraw/
## /mnt/genericStage/

In [0]:
# dbutils.fs.ls('/mnt/genericStage/2-WheelSales')

[FileInfo(path='dbfs:/mnt/genericStage/2-WheelSales/2-WheelSData.csv', name='2-WheelSData.csv', size=2085500, modificationTime=1719815771000)]

In [0]:
# dbutils.fs.unmount('/mnt/genericStage/4-WheelSData/')

/mnt/genericStage/4-WheelSData/ has been unmounted.


True

In [0]:
# df = spark.read.format('csv').option('Header', True).option('inferSchema', True).load('dbfs:/mnt/genericStage/2-WheelSales/')

In [0]:
# print(df.columns)

['OEM', 'Model', 'Segment', 'Month', 'Year', 'NoOfSales', 'EngineCapacity', 'Mileage', 'Model Price', 'Dealer location', 'Region']


In [0]:
# df.schema.fields

# print(df.dtypes)

[('OEM', 'string'), ('Model', 'string'), ('Segment', 'string'), ('Month', 'string'), ('Year', 'int'), ('NoOfSales', 'int'), ('EngineCapacity', 'string'), ('Mileage', 'string'), ('Model Price', 'string'), ('Dealer location', 'string'), ('Region', 'string')]


In [0]:
# schemas = df.dtypes

# types_in_list = [x[1] for x in schemas]

# print(types_in_list)

['string', 'string', 'string', 'string', 'int', 'int', 'string', 'string', 'string', 'string', 'string']


In [0]:
# result = df.select(*[trim(col(c)).alias(c) for c in df.columns]).show()

+-----+---------+----------+-----+----+---------+--------------------+-------+-------------------+---------------+------+
|  OEM|    Model|   Segment|Month|Year|NoOfSales|      EngineCapacity|Mileage|        Model Price|Dealer location|Region|
+-----+---------+----------+-----+----+---------+--------------------+-------+-------------------+---------------+------+
|Bajaj|  Platina|Motorcycle|  Jan|2022|    46492|                 100|  70-75|    52,166 - 63,578|          Delhi| North|
|Bajaj|       CT|Motorcycle|  Jan|2022|    12422|                 100|  75-80|    45,274 - 54,138|          Delhi| North|
|Bajaj|   CHETAK|EV Scooter|  Jan|2022|     1268|N/A (electric sco...|     85|1,00,000 - 1,15,000|          Delhi| North|
|Bajaj|       CT|Motorcycle|  Jan|2022|        0|                 125|  70-75|              55300|          Delhi| North|
|Bajaj|     Duke|Motorcycle|  Jan|2022|      778|                 125|  45-50|             150925|          Delhi| North|
|Bajaj|   Pulsar|Motorcy

In [0]:
# print(df.count())

347


In [0]:
# a = 'dbfs:/mnt/genericStage/2-WheelSales/'
# b = a.split('/')[3]
# print(b)

2-WheelSales


In [0]:
# dbutils.fs.ls('/mnt/genericStage/2-WheelSales/')

[FileInfo(path='dbfs:/mnt/genericStage/2-WheelSales/2-WheelSData.csv', name='2-WheelSData.csv', size=2085500, modificationTime=1719815771000)]

In [0]:
dt_str = datetime.now().strftime('%Y/%m/%d')
print(dt_str)

2024/07/03


In [0]:

def fn_handlenull_spaces():

    file_info = dbutils.fs.ls('/mnt/genericraw/')

    file_paths = [file.path for file in file_info]

    paths = [f"'{file.path}'" for file in file_info]

    for x in file_paths:
        print(x)
        full_path = x+dt_str
        print(full_path)
        dfs = spark.read.format('parquet').option('inferSchema', True).load(full_path)

        nulls_count = dfs.select([count( when (col(c).isNull() | isnan(c),c  )).alias(c) for c in dfs.columns  ]).first().asDict()

        display(nulls_count)

        total_nulls = 0
        for null_count in nulls_count.values():
            total_nulls += null_count
        print(f"Total null values in {x}: {total_nulls}")

        dfs_str = dfs.fillna('unkown', subset=dfs.schema.names)

        dfs_cleaned = dfs_str.fillna(0, subset=dfs.schema.names)

        nulls_count2 = dfs_cleaned.select([count( when (col(c).isNull() | isnan(c),c  )).alias(c) for c in dfs_cleaned.columns  ])

        display(nulls_count2)
        outfolder = x.split('/')[3]
        print(outfolder)

        dt_string = datetime.now().strftime("year=%Y/Month=%m/day=%d")
        
        print('writing')
        output_good_path = f"/mnt/genericSilver/{outfolder}/{dt_string}"

        dfs2 = dfs_cleaned.select(*[trim(col(c)).alias(c) for c in dfs_cleaned.columns])
        
        dfs2.coalesce(1).write.mode('overwrite').option("header", "true").parquet(output_good_path)
        print("Done")


In [0]:
fn_handlenull_spaces()

dbfs:/mnt/genericraw/2-WheelSales/
dbfs:/mnt/genericraw/2-WheelSales/2024/07/03


{'Dealerlocation': 0,
 'EngineCapacity': 960,
 'Mileage': 0,
 'Model': 2,
 'ModelPrice': 0,
 'Month': 0,
 'NoOfSales': 0,
 'OEM': 0,
 'Region': 0,
 'Segment': 2,
 'Year': 0}

Total null values in dbfs:/mnt/genericraw/2-WheelSales/: 964


Dealerlocation,EngineCapacity,Mileage,Model,ModelPrice,Month,NoOfSales,OEM,Region,Segment,Year
0,0,0,0,0,0,0,0,0,0,0


2-WheelSales
writing
Done
dbfs:/mnt/genericraw/4-WheelSales/
dbfs:/mnt/genericraw/4-WheelSales/2024/07/03


{'Brand': 0,
 'Model': 0,
 'BodyType': 0,
 'FUELTYPE': 0,
 'Mileage': 0,
 'ENGINE': 2160,
 'TRANSMISSION': 0,
 'Pricerange': 0,
 'FuelCapacity': 1800,
 'SeatingCapacity': 0,
 'Top_Speed': 0,
 'NoOfCylinders': 0,
 'YearOfSale': 0,
 'MonthOfSale': 0,
 'Country': 0,
 'City': 0}

Total null values in dbfs:/mnt/genericraw/4-WheelSales/: 3960


Brand,Model,BodyType,FUELTYPE,Mileage,ENGINE,TRANSMISSION,Pricerange,FuelCapacity,SeatingCapacity,Top_Speed,NoOfCylinders,YearOfSale,MonthOfSale,Country,City
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


4-WheelSales
writing
Done


In [0]:
control_df.show()

+------------------+-------------+-------------+-------------------+------------------+-----------------+--------------------+--------------------+--------------------+----------+--------+
|      PipelineName|Pk_PipelineID|Pk_ActivityID|PipelineSourceTypes|ParameterValueSink|ColumnSourceTypes|    SourceColumnName|    TargetColumnName|   sourcecolumntypes|IsNullable|IsActive|
+------------------+-------------+-------------+-------------------+------------------+-----------------+--------------------+--------------------+--------------------+----------+--------+
|PL_Copy2WheelsData|          101|            2|                CSV|  2-WheelSales.csv|              CSV|[OEM, Model, Segm...|[OEM, Model, Segm...|[string, string, ...|         0|       1|
|PL_Copy4WheelsData|          102|            3|                CSV|  4-WheelSales.csv|              CSV|[Brand, Model, Bo...|[Brand, Model, Bo...|[string, string, ...|      NULL|    NULL|
+------------------+-------------+-------------+-------

In [0]:
def schema_mappings(filepaths, reference_df):
    try:
        # Define a mapping for types
        type_dict = {
            'string': StringType(),
            'int': IntegerType(),
            'long': LongType(),
            'float': FloatType(),
            'double': DoubleType(),
            'boolean': BooleanType(),
            'date': DateType(),
            'timestamp': TimestampType()
        }

        # Iterate over each row in the reference DataFrame
        for row in reference_df.collect():
            pipeline_name = row['PipelineName']
            print(f"\nProcessing pipeline: {pipeline_name}")

            source_columns = row['SourceColumnName']
            target_columns = row['TargetColumnName']
            target_columns = target_columns.strip("[]").split(",")
            target_columns = [col_name.strip() for col_name in target_columns]

            source_types = row['sourcecolumntypes']
            source_types = source_types.strip("[]").split(",")
            source_types = [col_type.strip() for col_type in source_types]

            folder_names = row['test']

            print(f"Source Columns: {source_columns}")
            print(f"Target Columns: {target_columns}")
            print(f"Source Types: {source_types}")
            print(f"Folder Name: {folder_names}")

            if len(target_columns) != len(source_types):
                raise ValueError(f"Mismatch between target columns and source types lengths for {pipeline_name}")

            # Create the type mapping for the current pipeline
            type_mapping = {}
            for col_name, col_type in zip(target_columns, source_types):
                if col_type not in type_dict:
                    raise ValueError(f"Unsupported data type: {col_type}")
                type_mapping[col_name] = type_dict[col_type]

            print(f"Type Mapping: {type_mapping}")

            # Process the files for the current pipeline
            dt_string = datetime.now().strftime("year=%Y/Month=%m/day=%d")
            full_path = f"{filepaths}/{folder_names}/{dt_string}"

            print(f"Full Path: {full_path}")

            dfs = spark.read.format('parquet').load(full_path)
            actual_columns = dfs.columns
            print(f"Actual Columns: {actual_columns}")

            # Cast the columns to their appropriate types
            for col_name, col_type in type_mapping.items():
                if col_name in actual_columns:
                    print(col_name)
                    dfs = dfs.withColumn(col_name, col(col_name).cast(col_type))
                else:
                    raise ValueError(f"Column '{col_name}' from target columns does not exist in the DataFrame")

            # Print the updated schema
            dfs.printSchema()

            outfolder = folder_names
            print(outfolder)

            dt_string = datetime.now().strftime("year=%Y/Month=%m/day=%d")
        
            print('writing')
            output_good_path = f"/mnt/genericSilver/{outfolder}/{dt_string}"

            dfs.coalesce(1).write.mode('overwrite').option("header", "true").parquet(output_good_path)
            print("Done")

    except Exception as e:
        raise e

In [0]:
schema_mappings(filepaths='/mnt/genericSilver/', reference_df= res)


Processing pipeline: PL_Copy2WheelsData
Source Columns: [OEM, Model, Segment, Month, Year, NoOfSales, EngineCapacity, Mileage, Model Price, Dealer location, Region]
Target Columns: ['OEM', 'Model', 'Segment', 'Month', 'Year', 'NoOfSales', 'EngineCapacity', 'Mileage', 'ModelPrice', 'Dealerlocation', 'Region']
Source Types: ['string', 'string', 'string', 'string', 'int', 'int', 'string', 'string', 'string', 'string', 'string']
Folder Name: 2-WheelSales
Type Mapping: {'OEM': StringType(), 'Model': StringType(), 'Segment': StringType(), 'Month': StringType(), 'Year': IntegerType(), 'NoOfSales': IntegerType(), 'EngineCapacity': StringType(), 'Mileage': StringType(), 'ModelPrice': StringType(), 'Dealerlocation': StringType(), 'Region': StringType()}
Full Path: /mnt/genericSilver//2-WheelSales/year=2024/Month=07/day=03
Actual Columns: ['Dealerlocation', 'EngineCapacity', 'Mileage', 'Model', 'ModelPrice', 'Month', 'NoOfSales', 'OEM', 'Region', 'Segment', 'Year']
OEM
Model
Segment
Month
Year
N

In [0]:
dbutils.fs.ls('/mnt/genericSilver/2-WheelSales/year=2024/Month=07/day=03/')

[FileInfo(path='dbfs:/mnt/genericSilver/2-WheelSales/year=2024/Month=07/day=03/_committed_2437260535246933528', name='_committed_2437260535246933528', size=220, modificationTime=1720014033000),
 FileInfo(path='dbfs:/mnt/genericSilver/2-WheelSales/year=2024/Month=07/day=03/_committed_4642103600012763668', name='_committed_4642103600012763668', size=220, modificationTime=1720013746000),
 FileInfo(path='dbfs:/mnt/genericSilver/2-WheelSales/year=2024/Month=07/day=03/_committed_5004893812437007468', name='_committed_5004893812437007468', size=221, modificationTime=1719993804000),
 FileInfo(path='dbfs:/mnt/genericSilver/2-WheelSales/year=2024/Month=07/day=03/_committed_5154617478946274281', name='_committed_5154617478946274281', size=231, modificationTime=1719987743000),
 FileInfo(path='dbfs:/mnt/genericSilver/2-WheelSales/year=2024/Month=07/day=03/_committed_7551987012282441069', name='_committed_7551987012282441069', size=221, modificationTime=1720014113000),
 FileInfo(path='dbfs:/mnt/gene

In [0]:
df = spark.read.format('parquet').load('dbfs:/mnt/genericSilver/2-WheelSales/year=2024/Month=07/day=03/')

In [0]:
df.printSchema()

root
 |-- Dealerlocation: string (nullable = true)
 |-- EngineCapacity: string (nullable = true)
 |-- Mileage: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- ModelPrice: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- NoOfSales: integer (nullable = true)
 |-- OEM: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Year: integer (nullable = true)



In [0]:
# test(filepaths='/mnt/genericSilver/', reference_df= res)


Processing pipeline: PL_Copy2WheelsData
Source Columns: [OEM, Model, Segment, Month, Year, NoOfSales, EngineCapacity, Mileage, Model Price, Dealer location, Region]
Target Columns: ['OEM', 'Model', 'Segment', 'Month', 'Year', 'NoOfSales', 'EngineCapacity', 'Mileage', 'ModelPrice', 'Dealerlocation', 'Region']
Source Types: ['string', 'string', 'string', 'string', 'int', 'int', 'string', 'string', 'string', 'string', 'string']
Folder Name: 2-WheelSales
Type Mapping: {'OEM': StringType(), 'Model': StringType(), 'Segment': StringType(), 'Month': StringType(), 'Year': IntegerType(), 'NoOfSales': IntegerType(), 'EngineCapacity': StringType(), 'Mileage': StringType(), 'ModelPrice': StringType(), 'Dealerlocation': StringType(), 'Region': StringType()}
Full Path: /mnt/genericSilver//2-WheelSales/year=2024/Month=07/day=03
Actual Columns: ['Dealerlocation', 'EngineCapacity', 'Mileage', 'Model', 'ModelPrice', 'Month', 'NoOfSales', 'OEM', 'Region', 'Segment', 'Year']
OEM
Model
Segment
Month
Year
N

In [0]:
df = spark.read.format('parquet').load('')

In [0]:
# def schema_mappings (filepaths, reference_df):
#     try:
#         type_dict = {
#             'string' : StringType(),
#             'int' : IntegerType(),
#             'long' : LongType(),
#             'float' : FloatType(),
#             'double' : DoubleType(),
#             'boolean' : BooleanType(),
#             'date' : DateType()
#         }

#         for row in reference_df.collect():

#             pipeline_name = row['PipelineName']
#             print("sourcecolumnNames")
#             source_columns = row['SourceColumnName']
#             target_columns = row['TargetColumnName']
#             target_columns = target_columns.strip("[]").split(",")
#             print(len(target_columns))
#             target_columns = [col_name.strip() for col_name in target_columns]
#             source_types = row['sourcecolumntypes']
#             folder_names = row['test']

#             print(pipeline_name)
#             print(f"target_columns are {target_columns}")
#             print(source_types)
#             print(folder_names)

#             # print(source_types.strip("[]").replace("'","").split(", "))
#             source_types = source_types.strip("[]").split(",")
#             print(len(source_types))

#             if len(target_columns) != len(source_types):
#                 raise ValueError(f"Mismatch between target columns and source types lengths for {pipeline_name}")

#             type_mapping = {}

#             for col_name, col_type in zip(target_columns, source_types):
#                 col_type = col_type.strip()
#                 print(col_type)
#                 if col_type not in type_dict:
#                     raise ValueError (f"Unsupported data type : {col_type}")

#                 type_mapping[col_name] = type_dict[col_type]

#             print(f"Type Mapping : {type_mapping}")

#             file_info = dbutils.fs.ls(filepaths)


#             file_paths = [file.path for file in file_info]

#             dt_string = datetime.now().strftime("year=%Y/Month=%m/day=%d")

#             for x in file_paths:
#                 print(x)
#                 full_path = x+dt_string
#                 print(full_path)

#                 dfs = spark.read.format('parquet').load(full_path)

#                 actual_columns = dfs.columns

#                 print(f"Actual Columns : {actual_columns}")

#                 for col_name, col_type in type_mapping.items():
#                     if col_name in actual_columns :
#                         print(col_name)
#                     else:
#                         raise ValueError (f"Column '{col_name}' from target columns does not exist in the DataFrame")


     
#     except Exception as e:
#         raise e 
        


In [0]:
a = 'dbfs:/mnt/genericSilver/2-WheelSales/year=2024/Month=07/day=03/part-00000-tid-5004893812437007468-4b44db16-090a-4869-85ab-8bf81bd26653-11-1-c000.snappy.parquet'

print(a.split('/')[3])

2-WheelSales


In [0]:
# schema_mappings(filepaths='/mnt/genericSilver/', reference_df= res)

sourcecolumnNames
11
PL_Copy2WheelsData
target_columns are ['OEM', 'Model', 'Segment', 'Month', 'Year', 'NoOfSales', 'EngineCapacity', 'Mileage', 'ModelPrice', 'Dealerlocation', 'Region']
[string, string, string, string, int, int, string, string, string, string, string]
2-WheelSales
11
string
string
string
string
int
int
string
string
string
string
string
Type Mapping : {'OEM': StringType(), 'Model': StringType(), 'Segment': StringType(), 'Month': StringType(), 'Year': IntegerType(), 'NoOfSales': IntegerType(), 'EngineCapacity': StringType(), 'Mileage': StringType(), 'ModelPrice': StringType(), 'Dealerlocation': StringType(), 'Region': StringType()}
dbfs:/mnt/genericSilver/2-WheelSales/
dbfs:/mnt/genericSilver/2-WheelSales/year=2024/Month=07/day=03
Actual Columns : ['Dealerlocation', 'EngineCapacity', 'Mileage', 'Model', 'ModelPrice', 'Month', 'NoOfSales', 'OEM', 'Region', 'Segment', 'Year']
OEM
Model
Segment
Month
Year
NoOfSales
EngineCapacity
Mileage
ModelPrice
Dealerlocation
Region


---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-2178238408620504>, line 1
----> 1 schema_mappings(filepaths='/mnt/genericSilver/', reference_df= res)

File <command-2178238408620503>, line 111, in schema_mappings(filepaths, reference_df)
     74         # if isinstance(source_types, str):
     75         #     source_types = source_types.strip("[]").split(",")
     76 
   (...)
    106     #         actual_columns = dfs.columns
    107     #         print(actual_columns)
    110 except Exception as e:
--> 111     raise e

File <command-2178238408620503>, line 70, in schema_mappings(filepaths, reference_df)
     68                     print(col_name)
     69                 else:
---> 70                     raise ValueError (f"Column '{col_name}' from target columns does not exist in the DataFrame")
     74         # if isinstance(source_types, str):
     75         #   

In [0]:
# file_info = dbutils.fs.ls('/mnt/genericStage/')

# file_paths = [file.path for file in file_info]

# paths = [f"'{file.path}'" for file in file_info]

# for x in file_paths:
#     print(x)
#     dfs = spark.read.format('csv').option('Header', True).option('inferSchema', True).load(x)

#     nulls_count = dfs.select([count( when (col(c).isNull() | isnan(c),c  )).alias(c) for c in dfs.columns  ]).first().asDict()

#     display(nulls_count)

#     total_nulls = 0
#     for null_count in nulls_count.values():
#         total_nulls += null_count
#     print(f"Total null values in {x}: {total_nulls}")

#     dfs_str = dfs.fillna('unkown', subset=dfs.schema.names)

#     dfs_cleaned = dfs_str.fillna(0, subset=dfs.schema.names)

#     nulls_count2 = dfs_cleaned.select([count( when (col(c).isNull() | isnan(c),c  )).alias(c) for c in dfs_cleaned.columns  ])

#     display(nulls_count2)
#     outfolder = x.split('/')[3]
#     print(outfolder)



#     dt_string = datetime.now().strftime("year=%Y/Month=%m/day=%d")
    
#     print('writing')
#     output_good_path = f"/mnt/genericSilver/{outfolder}/{dt_string}"

#     dfs2 = dfs_cleaned.select(*[trim(col(c)).alias(c) for c in dfs_cleaned.columns])
    
#     dfs2.coalesce(1).write.mode('overwrite').option("header", "true").parquet(output_good_path)
#     print("Done")







dbfs:/mnt/genericStage/2-WheelSales/


{'OEM': 0,
 'Model': 2,
 'Segment': 2,
 'Month': 0,
 'Year': 0,
 'NoOfSales': 0,
 'EngineCapacity': 960,
 'Mileage': 0,
 'Model Price': 0,
 'Dealer location': 0,
 'Region': 0}

Total null values in dbfs:/mnt/genericStage/2-WheelSales/: 964


OEM,Model,Segment,Month,Year,NoOfSales,EngineCapacity,Mileage,Model Price,Dealer location,Region
0,0,0,0,0,0,0,0,0,0,0


2-WheelSales
writing
Done
dbfs:/mnt/genericStage/4-WheelSales/


{'Brand': 0,
 'Model': 0,
 'Body Type': 0,
 'FUEL TYPE': 0,
 'Mileage': 0,
 'ENGINE': 2160,
 'TRANSMISSION': 0,
 'Price range': 0,
 'Fuel Capacity (L)': 1800,
 'Seating Capacity': 0,
 'Top_Speed in Km/h': 0,
 'No Of Cylinders': 0,
 'Year Of Sale': 0,
 'Month Of Sale ': 0,
 'Country': 0,
 'City': 0}

Total null values in dbfs:/mnt/genericStage/4-WheelSales/: 3960


Brand,Model,Body Type,FUEL TYPE,Mileage,ENGINE,TRANSMISSION,Price range,Fuel Capacity (L),Seating Capacity,Top_Speed in Km/h,No Of Cylinders,Year Of Sale,Month Of Sale,Country,City
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


4-WheelSales
writing
Done


In [0]:
# dbutils.fs.help()

dbutils.fs provides utilities for working with FileSystems. Most methods in
this package can take either a DBFS path (e.g., "/foo" or "dbfs:/foo"), or
another FileSystem URI.

For more info about a method, use dbutils.fs.help("methodName") .

In notebooks, you can also use the %fs shorthand to access DBFS. The %fs shorthand maps
straightforwardly onto dbutils calls. For example, "%fs head --maxBytes=10000 /file/path"
translates into "dbutils.fs.head("/file/path", maxBytes = 10000)".
 mount mount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Mounts the given source directory into DBFS at the given mount point mounts: Seq -> Displays information about what is mounted within DBFS refreshMounts: boolean -> Forces all machines in this cluster to refresh their mount cache, ensuring they receive the most recent information unmount(mountPoint: String): boolean -> Deletes a DBFS mount point updateMount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Similar to mount(), but updates an existing mount point (if present) instead of creating a new one fsutils cp(from: String, to: String, recurse: boolean = false): boolean -> Copies a file or directory, possibly across FileSystems head(file: String, maxBytes: int = 65536): String -> Returns up to the first 'maxBytes' bytes of the given file as a String encoded in UTF-8 ls(dir: String): Seq -> Lists the contents of a directory mkdirs(dir: String): boolean -> Creates the given directory if it does not exist, also creating any necessary parent directories mv(from: String, to: String, recurse: boolean = false): boolean -> Moves a file or directory, possibly across FileSystems put(file: String, contents: String, overwrite: boolean = false): boolean -> Writes the given String out to a file, encoded in UTF-8 rm(dir: String, recurse: boolean = false): boolean -> Removes a file or directory

In [0]:
# dbutils.fs.unmount('/mnt/xyenta-dev-96-stg/People')

/mnt/xyenta-dev-96-stg/People has been unmounted.


True

In [0]:
# dbutils.fs.unmount('/mnt/Dev96Stage/Returns')

/mnt/Dev96Stage/Returns has been unmounted.


True